## Depencies

In [ ]:
%pip install numpy
%pip install matplotlib
%pip install ipympl

from math import *
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

import sys
sys.path.append('../src')
from leg_kinematics import LegKinematics

## Body inverse kinematics

1. Goal:

- Given the position of the end effectors in world frame and the body orientation, find the position of the points in shoulder frames.

2. Coordinates:

- World Frame
- Body Frame
- Shoulder Frame

3. Transformation Matrices:

- These are 4x4 homogeneous matrices that combine rotation (3x3 matrix) and translation (position vector) to describe how one coordinate frame relates to another.

- The equation shows how to transform from shoulder frame (s) to world frame (w) via the body frame (b).

$$
T_s = T_b T_s^b
$$

4. Steps:

- **Body to World**, $T_b$: Find the transformation matrix from body frame to world frame, using the orientation of the body (roll, pitch, yaw). 

- **Shoulder to Body**, $T_s^b$: Find the transformation matrix from shoulder frame to Body frame. It is determined by the robot’s mechanical design, and remains constant all the time.

- **Shoulder to World**, $T_s$: Find the transformation matrix from shoulder frame to world frame using the previous two matrices.

$$
T_s = T_b T_s^b
$$

- **World to Shoulder**, $T_w^s$, Find the inverse transformation matrix of $T_s$.

$$
T_w^s = inverse(T_s)
$$

- Project the point in world frame to shoulder frame

$$
p^s = T_w^s p^w
$$

For more details on coordinate transformations, see
[Coordinate Transformations](https://motion.cs.illinois.edu/RoboticSystems/CoordinateTransformations.html)

In [2]:
class QuadModel:
  def __init__(self):
    # Body dimensions, shoulder on the vertices
    body_width = 20
    body_length = 40
    self.body_z = 17

    # Foothold coordinates in world frame
    self.footholds = [
      np.array([body_length/2-4, (body_width/2), 0]),  # Front-left
      np.array([body_length/2-4, -(body_width/2), 0]), # Front-right
      np.array([-body_length/2+4, (body_width/2), 0]), # Back-left
      np.array([-body_length/2+4, -(body_width/2), 0]) # Back-right
    ]

    # Shoulder coordinates in body frame
    self.shoulders = [
      np.array([body_length/2, body_width/2, 0]),  # Front-left
      np.array([body_length/2, -body_width/2, 0]), # Front-right
      np.array([-body_length/2, body_width/2, 0]), # Back-left
      np.array([-body_length/2, -body_width/2, 0]) # Back-right
    ]

    # Transformation matrix
    # Naming convention:
    #   b: body frame
    #   s: shoulder frame

    # Body to World transformation
    self.Tb = self.calc_transform_matrix([0, 0, 0], [0, 0, self.body_z])
    
    # Rotation for shoulder frames to align with the body frame
    Rbs_front = self.calc_rotation_matrix(0, -np.pi, 0) @ self.calc_rotation_matrix(-np.pi/2, 0, 0)
    Rbs_back = self.calc_rotation_matrix(np.pi/2, 0, 0)

    # Shoulder to Body transformation, remain constant all the time
    self.Tbs_lst = []
    for shoulder in self.shoulders:
      Tbs = np.eye(4)
      Tbs[:3, :3] = Rbs_front if shoulder[0] > 0 else Rbs_back
      Tbs[:3, 3] = shoulder
      self.Tbs_lst.append(Tbs)

    # Initialize the legs and calculate the end effector points
    self.legs = [LegKinematics() for _ in range(4)]
    for (i, foothold) in enumerate(self.footholds):
      Ts = self.Tb @ self.Tbs_lst[i]
      Ts_inv = self.calc_inv_T(Ts)
      pe = self.calc_projected_point(Ts_inv, foothold)
      self.legs[i].set_ee_point(pe)

  def calc_rotation_matrix(self, roll, pitch, yaw):
    """
    Compute 3D rotation matrix from roll, pitch, yaw (XYZ Euler angles).
    """
    Rx = np.array([[1, 0, 0],
                    [0, np.cos(roll), -np.sin(roll)],
                    [0, np.sin(roll), np.cos(roll)]])
    
    Ry = np.array([[np.cos(pitch), 0, np.sin(pitch)],
                    [0, 1, 0],
                    [-np.sin(pitch), 0, np.cos(pitch)]])
    
    Rz = np.array([[np.cos(yaw), -np.sin(yaw), 0],
                    [np.sin(yaw), np.cos(yaw), 0],
                    [0, 0, 1]])

    return Rz @ Ry @ Rx

  def calc_transform_matrix(self, orientation, position):
    """
    Compute 4x4 transformation matrix from roll, pitch, yaw (XYZ Euler angles) and position for 
    current frame to align with the target frame. 

    @param orientation: Roll, pitch, yaw angles for current frame to align with the target frame
    @param position: Position of the current frame's origin in the target frame's coordinates
    """
    [roll, pitch, yaw] = orientation
    R = self.calc_rotation_matrix(roll, pitch, yaw)
    T = np.eye(4)
    T[:3, :3] = R
    T[:3, 3] = position
    return T
  
  def calc_inv_T(self, T):
    """
    Compute the inverse of the transformation matrix.
    """
    R = T[:3, :3]
    p = T[:3, 3]

    R_inv = R.T
    p_inv = -R_inv @ p

    T_inv = np.eye(4)
    T_inv[:3, :3] = R_inv
    T_inv[:3, 3] = p_inv

    return T_inv

  def calc_projected_point(self, transform_matrix, point):
    """
    Project a point to the new frame using the transformation matrix.

    @param transform_matrix: 4x4 transformation matrix
    @param point: Point to be projected
    """
    point = np.append(point, 1)
    projected_point = transform_matrix @ point
    return projected_point[:3]
  

In [3]:
def update_body_pose(self, roll, pitch, yaw):
  """
  Update the orientation and position of the body
  """
  
  # Body to World transformation
  self.Tb = self.calc_transform_matrix([roll, pitch, yaw], [0, 0, self.body_z])
  
  # Shoulder to World transformation
  for (i, foothold) in enumerate(self.footholds):
    Ts = self.Tb @ self.Tbs_lst[i]
    Ts_inv = self.calc_inv_T(Ts)
    pe = self.calc_projected_point(Ts_inv, foothold)
    self.legs[i].set_ee_point(pe)

QuadModel.update_body_pose = update_body_pose


In [4]:
def plot_transform_list(self, ax, axis_length=5.0):
    """
    Plot a list of transformation matrices in 3D.
    
    Args:
        T_list: list of 4x4 numpy arrays
        titles: list of str, labels for each frame
        axis_length: float, length of axes
    """

    titles = ["Front-Right", "Front-Left", "Rear-Right", "Rear-Left"]
    
    Ts_lst = []
    for i, (Tbs, title) in enumerate(zip(self.Tbs_lst, titles)):
        Ts = self.Tb @ Tbs
        p = Ts[:3, 3]
        R = Ts[:3, :3]
        Ts_lst.append(Ts)

        # Define axes
        x_axis = R @ np.array([axis_length, 0, 0])
        y_axis = R @ np.array([0, axis_length, 0])
        z_axis = R @ np.array([0, 0, axis_length])

        # Plot shoulder
        ax.scatter([p[0]], [p[1]], [p[2]], s=100, label=title)

        # Plot the leg
        points_world = []
        for point in self.legs[i].get_points():
            point_world = self.calc_projected_point(Ts, point)
            points_world.append(point_world)
        points = np.array(points_world)
        ax.plot3D(points[[0, 1, 2, 3], 0], points[[0, 1, 2, 3], 1], points[[0, 1, 2, 3], 2], 'blue')
        ax.plot3D(points[[2, 4, 5], 0], points[[2, 4, 5], 1], points[[2, 4, 5], 2], 'blue')

        # Plot axes
        ax.quiver(p[0], p[1], p[2], x_axis[0], x_axis[1], x_axis[2], 
                  color='red', linewidth=1)
        ax.quiver(p[0], p[1], p[2], y_axis[0], y_axis[1], y_axis[2], 
                  color='green', linewidth=1)
        ax.quiver(p[0], p[1], p[2], z_axis[0], z_axis[1], z_axis[2], 
                  color='blue', linewidth=1)
        
        # Plot the desired footholds
        ax.scatter([self.footholds[i][0]], [self.footholds[i][1]], [self.footholds[i][2]], 
                       color='black', s=50)
        
    shoulders = np.array([T[:3, 3] for T in Ts_lst])
    ax.plot3D(shoulders[[0, 1, 3, 2, 0], 0], shoulders[[0, 1, 3, 2, 0], 1], shoulders[[0, 1, 3, 2, 0], 2], 'blue')

    
    # Set labels and limits
    ax.set_xlabel('X (cm)')
    ax.set_ylabel('Y (cm)')
    ax.set_zlabel('Z (cm)')
    ax.set_title("Body Inverse Kinematics")

    ax.set_xlim((-30, 30))
    ax.set_ylim((-20, 20))
    ax.set_zlim((0, 20))
    ax.set_box_aspect([1, 1, 1])  # Equal aspect ratio (1:1:1)
    ax.view_init(azim=40)
    ax.legend(bbox_to_anchor=(1.04, 1), loc="upper left")

QuadModel.plot_transform_list = plot_transform_list


### Rotate around roll

In [ ]:
# Setup plot
fig = plt.figure(figsize=(7, 7))
ax = fig.add_subplot(111, projection='3d')  # Full figure for 3D plot

robot = QuadModel()
robot.update_body_pose(0, 0, 0)  # Initial pose
robot.plot_transform_list(ax)

# Animation parameters
frames = 60  # Number of frames
t = np.linspace(0, 2 * np.pi, frames)  # Time parameter for one cycle

def update(frame):
    ax.cla()  # Clear previous frame

    # Define oscillating motion
    roll = 0.05 * np.pi * np.sin(t[frame]) 
    pitch = 0 # 0.04 * np.pi * np.sin(t[frame] + np.pi/2)
    yaw = 0 #0.03 * np.pi * np.cos(t[frame])
    robot.update_body_pose(roll, pitch, yaw)
    robot.plot_transform_list(ax)

# Create animation
ani = FuncAnimation(fig, update, frames=frames, interval=20, repeat=True)
ax.set_xlim((-30, 30))
ax.set_ylim((-20, 20))
ax.set_zlim((0, 20))
HTML(ani.to_jshtml())
# ani.save('quad_animation.mp4')

### Rotate around pitch

In [ ]:
# Setup plot
fig = plt.figure(figsize=(7, 7))
ax = fig.add_subplot(111, projection='3d')  # Full figure for 3D plot

robot = QuadModel()
robot.update_body_pose(0, 0, 0)  # Initial pose
robot.plot_transform_list(ax)

# Animation parameters
frames = 60  # Number of frames
t = np.linspace(0, 2 * np.pi, frames)  # Time parameter for one cycle

def update(frame):
    ax.cla()  # Clear previous frame

    # Define oscillating motion
    roll = 0 #0.05 * np.pi * np.sin(t[frame]) 
    pitch = 0.04 * np.pi * np.sin(t[frame] + np.pi/2)
    yaw = 0 #0.03 * np.pi * np.cos(t[frame])
    robot.update_body_pose(roll, pitch, yaw)
    robot.plot_transform_list(ax)

# Create animation
ani = FuncAnimation(fig, update, frames=frames, interval=20, repeat=True)
ax.set_xlim((-30, 30))
ax.set_ylim((-20, 20))
ax.set_zlim((0, 20))
HTML(ani.to_jshtml())
# ani.save('quad_animation.mp4')


### Rotate around yaw

In [ ]:
# Setup plot
fig = plt.figure(figsize=(7, 7))
ax = fig.add_subplot(111, projection='3d')  # Full figure for 3D plot

robot = QuadModel()
robot.update_body_pose(0, 0, 0)  # Initial pose
robot.plot_transform_list(ax)

# Animation parameters
frames = 60  # Number of frames
t = np.linspace(0, 2 * np.pi, frames)  # Time parameter for one cycle

def update(frame):
    ax.cla()  # Clear previous frame

    # Define oscillating motion
    roll = 0 #0.05 * np.pi * np.sin(t[frame]) 
    pitch = 0 # 0.04 * np.pi * np.sin(t[frame] + np.pi/2)
    yaw = 0.03 * np.pi * np.cos(t[frame])
    robot.update_body_pose(roll, pitch, yaw)
    robot.plot_transform_list(ax)

# Create animation
# ani = FuncAnimation(fig, update, frames=frames, interval=20, repeat=True)
ax.set_xlim((-30, 30))
ax.set_ylim((-20, 20))
ax.set_zlim((0, 20))
HTML(ani.to_jshtml())
# ani.save('quad_animation.mp4')
